In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from uk.synth_uk import main

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
